In [ ]:
import uproot
import awkward as ak
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.patches import Ellipse
from IPython.display import Markdown, display

import awkward as ak

import networkx as nx
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram
from pylab import cm
import glob
import tqdm


%matplotlib inline

In [ ]:
def plot_tracksters(ax, vx, vy, vz, ve):
    ax.set_xlabel("x (cm)")
    ax.set_ylabel("y (cm)")
    ax.set_zlabel("z (cm)")
    if ve is None:
        ve = np.ones(len(vx))*10
    for i, x, y, z, e in zip(range(len(vx)), vx, vy, vz, ve):
        plot_trackster(ax, i, x, y, z, e)
def get_event_window(ex, ey, ez, ee, zoom=1, e_threshold=2):

    ef = np.where(ak.flatten(ee) > e_threshold)

    x_max = max(ak.flatten(ex)[ef])
    x_min = min(ak.flatten(ex)[ef])
    y_max = max(ak.flatten(ey)[ef])
    y_min = min(ak.flatten(ey)[ef])
    z_max = max(ak.flatten(ez)[ef])
    z_min = min(ak.flatten(ez)[ef])

    x_zoom = (x_max - x_min) / zoom
    x_max -= x_zoom
    x_min += x_zoom

    y_zoom = (y_max - y_min) / zoom
    y_max -= y_zoom
    y_min += y_zoom

    z_zoom = (z_max - z_min) / zoom
    z_max -= z_zoom
    z_min += z_zoom

    return (x_min, x_max), (y_min, y_max), (z_min, z_max)

In [ ]:
def check_point_cone(h,base_radius,angle,start_point,x,y,z,eigenvector):
    test_point = np.array([x,y,z])
    base_radius = np.tan(angle)*h
    cone_dist = np.dot(test_point-start_point,eigenvector)
    cone_r = (cone_dist/h)*base_radius
    dist = np.linalg.norm((test_point-start_point)-cone_dist*eigenvector)
    return dist<cone_r

def connected_components(pairs):
    components = []

    for a, b in pairs:
        for component in components:
            if a in component:
                for i, other_component in enumerate(components):
                    if b in other_component and other_component != component: 
                        component.extend(other_component)
                        components[i:i+1] = []
                        break 
                else: 
                    if b not in component:
                        component.append(b)
                break 
            if b in component: 
                component.append(a)
                break 
        else: 
#             if components:
#                 if a or b in component:
#                     break
#             else:
            components.append([a, b])
        
       
    return components
def flatten_array(arr):
    flattened = []
    for item in arr:
        if isinstance(item, list):
            flattened.extend(flatten_array(item))
        else:
            flattened.append(item)
    return flattened
def order_pairs(pairs):
    sorted_pairs = sorted(pairs, key=lambda pair: (pair[1], pair[0]))
    return sorted_pairs

In [ ]:
def computeRatio(df1, df2):
    ratio = df1/ df2
    ratio_st = np.nan_to_num(np.sqrt((np.sqrt(df1)/df1)**2 + (np.sqrt(df2)/df2)**2))
    return ratio, ratio_st

def recomputeError(h1, h2, NBINS):
    h1.Sumw2()
    h2.Sumw2()
    ratio = h1.Clone('h3')
    ratio.Sumw2()
    ratio.Divide(h2)
    bin_errors = []
    for i in range(ratio.GetSize()):
        bin_errors.append(ratio.GetBinError(i))
    return bin_errors

def fixError(ratio, bin_errors):
    bin_errors_asym_neg = []
    bin_errors_asym_pos = []
    for i,e in enumerate(bin_errors[1:-1]):
        div = 1
        bin_errors_asym_neg.append(e/div)
        if(e/div + ratio[i] > 1):
            bin_errors_asym_pos.append(1-ratio[i])
        else:
            bin_errors_asym_pos.append(e/div)
    return [bin_errors_asym_neg, bin_errors_asym_pos]


In [ ]:
filename = "/eos/cms/store/group/dpg_hgcal/comm_hgcal/wredjeb/TICLv4Sample/CMSSW_13_X/GNNTraining/CloseByPion0PU/ntuples_10_600/new_ntuples_15643818_42.root"
file = uproot.open(filename)
tracks = file["ticlNtuplizer/tracks"]
tracksters = file["ticlNtuplizer/tracksters"]
simtracksters = file["ticlNtuplizer/simtrackstersSC"]
simtrackstersCP = file["ticlNtuplizer/simtrackstersCP"]
ass = file["ticlNtuplizer/associations"]
gra = file["ticlNtuplizer/graph"]
cand = file["ticlNtuplizer/candidates"]
lc = file["ticlNtuplizer/clusters"]


In [ ]:
trk_data = tracksters.arrays(["NTracksters", "raw_energy","raw_em_energy","barycenter_x","barycenter_y",
                              "barycenter_z","eVector0_x", "eVector0_y","eVector0_z","EV1","EV2","EV3",
                              "vertices_indexes", "sigmaPCA1", "sigmaPCA2", "sigmaPCA3", "vertices_x", "vertices_y",
                              "vertices_z", "vertices_energy", "layer_cluster_seed","trackster_barycenter_eta","trackster_barycenter_phi","raw_pt","NClusters"])
gra_data = gra.arrays(["linked_inners", "linked_outers"])
# ass_data = ass.arrays(["tsCLUE3D_recoToSim_SC", "tsCLUE3D_recoToSim_SC_score",
#                       "tsCLUE3D_simToReco_SC", "tsCLUE3D_simToReco_SC_score",
#                        "tsCLUE3D_simToReco_SC_sharedE", "tsCLUE3D_recoToSim_SC_sharedE"])

ass_data = ass.arrays([#'tsCLUE3D_recoToSim_PU', 'tsCLUE3D_recoToSim_PU_score', 'tsCLUE3D_recoToSim_PU_sharedE',
                                                      'tsCLUE3D_recoToSim_SC', 'tsCLUE3D_recoToSim_SC_score', 'tsCLUE3D_recoToSim_SC_sharedE', 'tsCLUE3D_simToReco_SC', 'tsCLUE3D_simToReco_SC_score', 
                                                      'tsCLUE3D_simToReco_SC_sharedE', 'tsCLUE3D_recoToSim_CP', 'tsCLUE3D_recoToSim_CP_score', 'tsCLUE3D_recoToSim_CP_sharedE', 'tsCLUE3D_simToReco_CP',
                                                      'tsCLUE3D_simToReco_CP_score', 'tsCLUE3D_simToReco_CP_sharedE',#'Mergetracksters_recoToSim_PU', #'Mergetracksters_recoToSim_PU_score'
#                                                           'Mergetracksters_recoToSim_PU_sharedE', 'Mergetracksters_simToReco_PU', 'Mergetracksters_simToReco_PU_score', 'Mergetracksters_simToReco_PU_sharedE', 
                                                      'Mergetracksters_recoToSim_CP','Mergetracksters_recoToSim_CP_score','Mergetracksters_simToReco_CP', 'Mergetracksters_simToReco_CP_sharedE', 
                                                      'Mergetracksters_simToReco_CP_score'])

cand_data = cand.arrays(["tracksters_in_candidate"])
lc_data = lc.arrays(['position_x', 'position_y', 'position_z', 'energy'])
try:
    simtrackstersSC_data = simtracksters.arrays(["stsSC_raw_energy", 'stsSC_boundaryPx', 'stsSC_boundaryPy', 'stsSC_boundaryPz'])
    simtrackstersCP_data = simtrackstersCP.arrays(["stsCP_raw_energy", 'stsCP_boundaryPx', 'stsCP_boundaryPy', 'stsCP_boundaryPz',"trackster_barycenter_eta","trackster_barycenter_phi","raw_pt"])
    
except Exception:
    simtrackstersSC_data = simtracksters.arrays(["raw_energy", 'boundaryPx', 'boundaryPy', 'boundaryPz','barycenter_x','NTracksters'])
    simtrackstersCP_data = simtrackstersCP.arrays(["raw_energy", 'boundaryPx', 'boundaryPy', 'boundaryPz','barycenter_x','NTracksters',"trackster_barycenter_eta","trackster_barycenter_phi","raw_pt"])

In [ ]:
trk_data["vertices_indexes"][0][3]

In [ ]:
lc.keys()

In [ ]:
simtrackstersCP.keys()

In [ ]:

eid = 4
fig = plt.figure(figsize=(20, 20))
ax1 = fig.add_subplot(111, projection='3d')
ax1.set_xlabel("x (cm)")
ax1.set_ylabel("y (cm)")
# ax2 = fig.add_subplot(121, projection='3d')
# ax2.set_xlim(xlim)
# ax2.set_ylim(ylim)
# ax2.set_zlim(zlim)
# ax2.set_xlabel("x (cm)")
# ax2.set_ylabel("y (cm)")
# # ax1.set_zlabel("detector layer")
ax1.set_zlabel("z (cm)")
# ax2.set_zlabel("z (cm)")

# plt.axis('off')
trk_xb = trk_data["barycenter_x"][eid]
trk_yb = trk_data["barycenter_y"][eid]
trk_zb = trk_data["barycenter_z"][eid]
trk_e = trk_data["raw_energy"][eid]
trk_x = trk_data.vertices_x[eid]
trk_y = trk_data.vertices_y[eid]
trk_z = trk_data.vertices_z[eid]
trk_vi = trk_data["vertices_indexes"][eid]
ex = trk_data["eVector0_x"][eid]
ey = trk_data["eVector0_y"][eid]
ez= trk_data["eVector0_z"][eid]
evx = trk_data["EV1"][eid]
evy = trk_data["EV2"][eid]
evz = trk_data["EV3"][eid]
percent = 0.
angle = .33
trk_r = np.sqrt(trk_data["barycenter_x"][eid]**2+trk_data["barycenter_y"][eid]**2+trk_data["barycenter_z"][eid]**2)
max_r = np.max(trk_r)

points = []
edges = []
for vi_m,(x, y, z, e,eigenvector1,eigenvector2,eigenvector3,vi) in enumerate(zip(trk_xb, trk_yb, trk_zb, trk_e,ex,ey,ez,trk_vi)):
   #height for cone visualization
    h = 20
    #create vertex point 
    start_point = np.array([x,y,z])
    #define direction of trackster barycenter and normalize
    eigenvector = np.array([eigenvector1,eigenvector2,eigenvector3])
    eigenvector /=np.linalg.norm(eigenvector)
    #create arbitrary vector not in same direction as eigenvector in order to create orthogonal vector
    not_ev = np.array([1,0,0])
    if (eigenvector ==not_ev).all():
        not_ev = np.array([0,1,0])
    #create orthogonal vector 1
    ev_cross = np.cross(eigenvector,not_ev)
    #create orthogonal vector 2
    ev_cross/=np.linalg.norm(ev_cross)
    ev_cross2 = np.cross(eigenvector,ev_cross)
    #ev_cross2/=np.linalg.norm(ev_cross)
    #cone_dist = np.dot(test_point-start_point,eigenvector)

    #surface ranges over t from 0 to length of axis and 0 to 2*pi
    
    t = np.linspace(0, h, 100)
    theta = np.linspace(0, 2 * np.pi, 100)
    #use meshgrid to make 2d arrays, generate cone central axis and opening
    line_points = [start_point + t * eigenvector for t in t]
    base_radius = np.tan(angle)*(np.linalg.norm(line_points[-1]-start_point))

    line_points = np.array(line_points)
    cone_dist = np.dot(line_points-start_point,eigenvector)
    R = (cone_dist/h)*base_radius
    t, theta = np.meshgrid(t, theta)
    #search through all tracksters, creating an edge if it exists within cone of main tracksters
    data = [[l,x,y,z,j] for l,(x,y,z,j) in enumerate(zip(trk_xb, trk_yb, trk_zb,trk_vi))]
    ordered_data = sorted(data, key=lambda a: -a[2])
    for vi_n,xp, yp, zp, j in ordered_data:
        if check_point_cone(h,base_radius,angle,start_point,xp,yp,zp,eigenvector):
            edges.append([vi_m,vi_n])
            ax1.plot([xp, x], [yp, y], [zp, z], c='blue', alpha=0.4)
    
    #generate coordinates for surface
    X, Y, Z = [start_point[i] + eigenvector[i] * t + R* np.sin(theta) * ev_cross[i] + R* np.cos(theta) * ev_cross2[i] for i in [0, 1, 2]]
    ax1.plot_surface(X, Y, Z,alpha=.3)
    ax1.scatter(x, y, z, s=20*e)
    #ax2.plot_surface(X, Y, Z,alpha=1)
   # ax2.scatter(x, y, z, s=20*e)

clusters_x = lc["position_x"].array()
clusters_y = lc["position_y"].array()
clusters_z = lc["position_z"].array()
clusters_e = lc["energy"].array()
vi = trk_data.vertices_indexes[eid]
# vi = simtracksters["stsSC_vertices_indexes"].array()[0:eid]

# only take one endcap
vz = ak.Array([clusters_z[eid][indices] for indices in vi])
# tdx = vz > 0

vx = ak.Array([clusters_x[eid][indices] for indices in vi])
vy = ak.Array([clusters_y[eid][indices] for indices in vi])
ve = ak.Array([clusters_e[eid][indices] for indices in vi])

vzm = vz

xlim, ylim, zlim = get_event_window(vx, vy, vzm, ve)
ax1.set_xlim(xlim)
ax1.set_ylim(ylim)
ax1.set_zlim(zlim)
ax1.set_title(f"Top view: Trackster cones from PCA axis ({angle}) radians")
ax1.view_init(0,180)
#ax2.set_title(f"Front view: Trackster cones from PCA axis ({angle}) radians")
#ax2.view_init(0, 90)
plt.show()
ax1.get_yaxis().set_ticks([])
ax1.get_yaxis().set_visible(False)

In [ ]:
path = "/eos/cms/store/group/dpg_hgcal/comm_hgcal/wredjeb/TICLv4Sample/CMSSW_13_X/GNNTraining/CloseByPion0PU/ntuples_10_600/"
files = glob.glob(path + "*.root") 
trk_data_tot = []
gra_data_tot = []
ass_data_tot = []
lc_data_tot = []
simtrackstersSC_data_tot = []
simtrackstersCP_data_tot = []
for f in files:
    file = uproot.open(f)
    
    try:
        tracks = file["/ticlNtuplizer/tracks;1"]
        tracksters = file["ticlNtuplizer/tracksters;1"]
        simtracksters = file["ticlNtuplizer/simtrackstersSC;1"]
        simtrackstersCP = file["ticlNtuplizer/simtrackstersCP;1"]
        ass = file["ticlNtuplizer/associations;1"]
        gra = file["ticlNtuplizer/graph;1"]
        cand = file["ticlNtuplizer/candidates;1"]
        lc = file["ticlNtuplizer/clusters;1"]
        
    except Exception:
        continue
    trk_data = tracksters.arrays(["NTracksters", "raw_energy","raw_em_energy","barycenter_x","barycenter_y",
                              "barycenter_z","eVector0_x", "eVector0_y","eVector0_z","EV1","EV2","EV3",
                              "vertices_indexes", "sigmaPCA1", "sigmaPCA2", "sigmaPCA3", "vertices_x", "vertices_y",
                              "vertices_z", "vertices_energy", "layer_cluster_seed","trackster_barycenter_eta","trackster_barycenter_phi","raw_pt"])
    gra_data = gra.arrays(["linked_inners", "linked_outers"])
    # ass_data = ass.arrays(["tsCLUE3D_recoToSim_SC", "tsCLUE3D_recoToSim_SC_score",
    #                       "tsCLUE3D_simToReco_SC", "tsCLUE3D_simToReco_SC_score",
    #                        "tsCLUE3D_simToReco_SC_sharedE", "tsCLUE3D_recoToSim_SC_sharedE"])

    ass_data = ass.arrays([#'tsCLUE3D_recoToSim_PU', 'tsCLUE3D_recoToSim_PU_score', 'tsCLUE3D_recoToSim_PU_sharedE',
                                                          'tsCLUE3D_recoToSim_SC', 'tsCLUE3D_recoToSim_SC_score', 'tsCLUE3D_recoToSim_SC_sharedE', 'tsCLUE3D_simToReco_SC', 'tsCLUE3D_simToReco_SC_score', 
                                                          'tsCLUE3D_simToReco_SC_sharedE', 'tsCLUE3D_recoToSim_CP', 'tsCLUE3D_recoToSim_CP_score', 'tsCLUE3D_recoToSim_CP_sharedE', 'tsCLUE3D_simToReco_CP',
                                                          'tsCLUE3D_simToReco_CP_score', 'tsCLUE3D_simToReco_CP_sharedE',#'Mergetracksters_recoToSim_PU', #'Mergetracksters_recoToSim_PU_score'
    #                                                           'Mergetracksters_recoToSim_PU_sharedE', 'Mergetracksters_simToReco_PU', 'Mergetracksters_simToReco_PU_score', 'Mergetracksters_simToReco_PU_sharedE', 
                                                          'Mergetracksters_recoToSim_CP','Mergetracksters_recoToSim_CP_score','Mergetracksters_simToReco_CP', 'Mergetracksters_simToReco_CP_sharedE', 
                                                          'Mergetracksters_simToReco_CP_score'])

    cand_data = cand.arrays(["tracksters_in_candidate"])
    lc_data = lc.arrays(['position_x', 'position_y', 'position_z', 'energy','cluster_number_of_hits'])
    lc_data_tot.append(lc_data)
    ass_data_tot.append(ass_data)
    trk_data_tot.append(trk_data)
    gra_data_tot.append(gra_data)
    
    try:
        simtrackstersSC_data = simtracksters.arrays(["stsSC_raw_energy", 'stsSC_boundaryPx', 'stsSC_boundaryPy', 'stsSC_boundaryPz'])
        simtrackstersCP_data = simtrackstersCP.arrays(["stsCP_raw_energy", 'stsCP_boundaryPx', 'stsCP_boundaryPy', 'stsCP_boundaryPz',"trackster_barycenter_eta","trackster_barycenter_phi","raw_pt","vertices_indexes"])

    except Exception:
        simtrackstersSC_data = simtracksters.arrays(["raw_energy", 'boundaryPx', 'boundaryPy', 'boundaryPz','barycenter_x','NTracksters'])
        simtrackstersCP_data = simtrackstersCP.arrays(["raw_energy", 'boundaryPx', 'boundaryPy', 'boundaryPz','barycenter_x','NTracksters',"trackster_barycenter_eta","trackster_barycenter_phi","raw_pt","vertices_indexes"])
    simtrackstersCP_data_tot.append(simtrackstersCP_data)
    simtrackstersSC_data_tot.append(simtrackstersSC_data)

In [ ]:
points = []
edges_tot = []
#angle =.33 #np.pi/2
angle = np.pi/6
for f in range(len(trk_data_tot)):
    edges_ev = []
    for ev in range(len(trk_data_tot[f])):
        points = []
        edges = []    
        trk_xb = trk_data_tot[f]["barycenter_x"][ev]
        trk_yb = trk_data_tot[f]["barycenter_y"][ev]
        trk_zb = trk_data_tot[f]["barycenter_z"][ev]
        trk_e = trk_data_tot[f]["raw_energy"][ev]
        trk_x = trk_data_tot[f].vertices_x[ev]
        trk_y = trk_data_tot[f].vertices_y[ev]
        trk_z = trk_data_tot[f].vertices_z[ev]
        trk_vi = trk_data_tot[f]["vertices_indexes"][ev]
        ex = trk_data_tot[f]["eVector0_x"][ev]
        ey = trk_data_tot[f]["eVector0_y"][ev]
        ez= trk_data_tot[f]["eVector0_z"][ev]
        for vi_m,(x, y, z, e,eigenvector1,eigenvector2,eigenvector3,vi) in enumerate(zip(trk_xb, trk_yb, trk_zb, trk_e,ex,ey,ez,trk_vi)):
           #height for cone visualization
            h = 20
            t = np.linspace(0, h, 100)

            #create vertex point 
            start_point = np.array([x,y,z])
            #define direction of trackster barycenter and normalize
            eigenvector = np.array([eigenvector1,eigenvector2,eigenvector3])
            eigenvector /=np.linalg.norm(eigenvector)
            line_points = [start_point + t * eigenvector for t in t]
            base_radius = np.tan(angle)*(np.linalg.norm(line_points[-1]-start_point))

            #create arbitrary vector not in same direction as eigenvector in order to create orthogonal vector
            #search through all tracksters, creating an edge if it exists within cone of main tracksters
            data = [[l,x,y,z,j] for l,(x,y,z,j) in enumerate(zip(trk_xb, trk_yb, trk_zb,trk_vi))]
            ordered_data = sorted(data, key=lambda a: -a[2])
            for vi_n,xp, yp, zp, j in ordered_data:
                if check_point_cone(h,base_radius,angle,start_point,xp,yp,zp,eigenvector):
                    edges.append([vi_m,vi_n])
        edges_ev.append(edges)
    edges_tot.append(edges_ev)
        


In [ ]:
lc_en[sim_lc_indices[e][0]]

In [ ]:
threshold = .8
eta_num_fill = []
energy_num_fill = []
pt_num_fill = [] 
phi_num_fill = []
eta_den_fill = []
energy_den_fill = []
pt_den_fill = [] 
phi_den_fill = []
num = 0
den = 0
for f in range(len(trk_data_tot)):
    for ev in range(len(trk_data_tot[f])):
        simToRecoCP_indices = ass_data_tot[f].tsCLUE3D_simToReco_CP[ev]
        simToRecoCP_sharedFraction = ass_data_tot[f].tsCLUE3D_simToReco_CP_sharedE[ev]
        simToRecoCP_score = ass_data_tot[f].tsCLUE3D_simToReco_CP_score[ev]
        sim_energy = simtrackstersCP_data_tot[f].raw_energy[ev]
        sim_pt = simtrackstersCP_data_tot[f].raw_pt[ev]
        sim_phi = simtrackstersCP_data_tot[f].trackster_barycenter_phi[ev]
        sim_eta = simtrackstersCP_data_tot[f].trackster_barycenter_eta[ev]
        reco_energy = trk_data_tot[f].raw_energy[ev]
        reco_pt = trk_data_tot[f].raw_pt[ev]
        reco_phi = trk_data_tot[f].trackster_barycenter_phi[ev]
        reco_eta = trk_data_tot[f].trackster_barycenter_eta[ev]
        lc_en = lc_data_tot[f].energy[ev]
        lc_num_hits = lc_data_tot[f].cluster_number_of_hits[ev]
        sim_lc_indices = simtrackstersCP_data_tot[f].vertices_indexes[ev]
        

        tracksters_ev = trk_data_tot[f][ev]
        
        edges = edges_tot[f][ev]
        connected_edges = connected_components(order_pairs(edges))

        #print("edges")
       # print(edges)

        #for each simtrackster in the event
        reco_trk_energies = []
        reco_trk_indices = []
        efficiencies = []
        sim_track_energies = []
        sim_numerator = []
        sim_denominator = []
        for e in range(len(simToRecoCP_indices)):
            isMatched = False
            corrected_sim_energy = np.sum([lc_en[b] for b in  sim_lc_indices[e] if lc_num_hits[b]>1])
            num_lcs = [lc_en[b] for b in  sim_lc_indices[e] if lc_num_hits[b]>1]
            sim_track_energies.append(corrected_sim_energy)

            for r in range(len(connected_edges)):
                recoindices = [list(simToRecoCP_indices[e]).index(b) for b in connected_edges[r]]
                reco_trk_energies.append(np.sum(simToRecoCP_sharedFraction[0][recoindices]))
                efficiency = np.sum(simToRecoCP_sharedFraction[0][recoindices])/corrected_sim_energy
               # print("ratio of passed LCs vs total", len(num_lcs)/len(sim_lc_indices[e]))
               # efficiencycorrected = np.sum(simToRecoCP_sharedFraction[0][recoindices])/np.sum(reco_energy[recoindices])
                #efficiency_sim = np.sum(simToRecoCP_sharedFraction[0][recoindices])/sim_energy[e]
               # print("efficiency",efficiency, "corr", efficiencycorrected,"sim",efficiency_sim)

               # if f<2 and ev<10:
               # print(corrected_sim_energy,"  ",np.sum(reco_energy[recoindices]))
                sim_denominator.append([sim_energy[e],sim_pt[e],sim_phi[e],sim_eta[e]])
                if efficiency>threshold:
                    num+=1
                    isMatched= True
                    sim_numerator.append([sim_energy[e],sim_pt[e],sim_phi[e],sim_eta[e]])
                    if sim_pt[e]>10.:
                        eta_num_fill.append(sim_numerator[e][3])
                    phi_num_fill.append(sim_numerator[e][2])
                    energy_num_fill.append(sim_numerator[e][0])
                    pt_num_fill.append(sim_numerator[e][1])
                if sim_pt[e]>10:
                    eta_den_fill.append(sim_denominator[e][3])
                phi_den_fill.append(sim_denominator[e][2])
                energy_den_fill.append(sim_denominator[e][0])
                pt_den_fill.append(sim_denominator[e][1])
                den+=1
print(num/den)
#given reconstruction that we have, want to quantify
#sum of shared_energy
#sum of the shared fractions of the recotracksters
#sum of the energies of the recotracksters that are connected
#sum of these energies
#of simtracksters #number of reconstrcuted simtracksters
#goes up by one when #event
#numerator is how many times we actually get this property\
#creat a self edge between all ndes
# edge
# given a graph, also compute energy that shouldn't o
#maximize efficiency minimize contamination


In [ ]:
NBINS = 10
NBINSPT = 10
NBINSPHI = 10
NBINSETA = 10
e_N, bins_e = np.histogram(energy_num_fill, bins = NBINS, range = (min(energy_den_fill), max(energy_den_fill)))
e_D, bins_e = np.histogram(energy_den_fill, bins = NBINS, range = (min(energy_den_fill), max(energy_den_fill)))
pt_N, bins_pt = np.histogram(pt_num_fill, bins = NBINSPT, range = (min(pt_den_fill),max(pt_den_fill)))
pt_D, bins_pt = np.histogram(pt_den_fill, bins = NBINSPT, range = (min(pt_den_fill), max(pt_den_fill)))
phi_N, bins_phi = np.histogram(phi_num_fill, bins = NBINSPHI, range = (min(phi_den_fill),max(phi_den_fill)))
phi_D, bins_phi = np.histogram(phi_den_fill, bins = NBINSPHI, range = (min(phi_den_fill), max(phi_den_fill)))
eta_N, bins_eta = np.histogram(eta_num_fill, bins = NBINSETA, range = (min(eta_den_fill),max(eta_den_fill)))
eta_D, bins_eta = np.histogram(eta_den_fill, bins = NBINSETA, range = (min(eta_den_fill), max(eta_den_fill)))

In [ ]:
ratioE, stdE = computeRatio(e_N,e_D)
#errorsE = fixError(ratioE, bin_errorsE)
ratioPt, stdPt = computeRatio(pt_N,pt_D)
#errorsPt = fixError(ratioPt, bin_errorsPt)
ratioEta, stdEta = computeRatio(eta_N,eta_D)
#errorsEta = fixError(ratioEta, bin_errorsEta)
ratioPhi, stdPhi = computeRatio(phi_N,phi_D)
#errorsPhi = fixError(ratioPhi, bin_errorsPhi)


In [ ]:
plt.figure(figsize = (20,5))
i_c = 0
col = 'blue'
Particle = "Pion"
PU = 0
c = .8
plt.subplot(1,4,1)
plt.errorbar(bins_e[:-1], ratioE, yerr=stdE, linestyle = None, marker = 'o', lw = 1, label = f"Shared Fraction Threshold {c}", capsize = 2, c =  col, markersize = 8)
# plt.errorbar(bins_e3[:-1], ratioE3, yerr=stdE3, linestyle = None, marker = 'o', lw = 1, label = 'TICLv3', capsize = 2)
plt.ylim(0,1.05)
if(i_c == 0):
    plt.suptitle("TracksterMerge - Efficiency ")
    plt.xlabel("gen Energy [GeV]")
    plt.ylabel("Efficiency")
    plt.annotate(f"Single {Particle} {PU} PU\nE $\in$ [10,600]\n$\eta$ $\in$ [1.7, 2.7]", xy = (0.4,0.4), xycoords='axes fraction')
    #hep.cms.text('Simulation')

plt.subplot(1,4,2)
plt.errorbar(bins_pt[:-1], ratioPt, yerr=stdPt, linestyle = None, marker = 'o', lw = 1, label = f"Shared Fraction Threshold {c}", capsize = 2, c = col, markersize = 8)
# plt.errorbar(bins_e3[:-1], ratioE3, yerr=stdE3, linestyle = None, marker = 'o', lw = 1, label = 'TICLv3', capsize = 2)
plt.ylim(0,1.05)
if(i_c == 0):
    plt.suptitle("TracksterMerge - Efficiency ")
    plt.xlabel("gen $p_{T}$ [GeV]")
    plt.ylabel("Efficiency")
    plt.annotate(f"Single {Particle} {PU} PU\nE $\in$ [10,600]\n$\eta$ $\in$ [1.7, 2.7]", xy = (0.4,0.4), xycoords='axes fraction')
    #hep.cms.text('Simulation')
plt.xlim(None, np.max(pt_den_fill))
plt.subplot(1,4,3)
plt.errorbar(bins_phi[:-1], ratioPhi, yerr=stdPhi, linestyle = None, marker = 'o', lw = 1, label = f"Shared Fraction Threshold {c}", capsize = 2, c = col, markersize = 8)
# plt.errorbar(bins_e3[:-1], ratioE3, yerr=stdE3, linestyle = None, marker = 'o', lw = 1, label = 'TICLv3', capsize = 2)
plt.ylim(0,1.05)
if(i_c == 0):
    plt.suptitle("TracksterMerge - Efficiency ")
    plt.xlabel("gen $\phi$")
    plt.ylabel("Efficiency")
    plt.annotate(f"Single {Particle} {PU} PU\nE $\in$ [10,600]\n$\eta$ $\in$ [1.7, 2.7]", xy = (0.4,0.4), xycoords='axes fraction')
    #hep.cms.text('Simulation')
plt.subplot(1,4,4)
plt.errorbar(bins_eta[:-1], ratioEta, yerr=stdEta, linestyle = None, marker = 'o', lw = 1, label = f"Shared Fraction Threshold {c}", capsize = 2, c = col, markersize = 8)
# plt.errorbar(bins_e3[:-1], ratioE3, yerr=stdE3, linestyle = None, marker = 'o', lw = 1, label = 'TICLv3', capsize = 2)
plt.ylim(0,1.05)
if(i_c == 0):
    plt.suptitle(f"ConeGraphv1 - Efficiency {angle} Radians")
    plt.xlabel("gen $\eta$ for $p_{T}$ >10 [GeV] ")
    plt.ylabel("Efficiency")
    plt.annotate(f"Single {Particle} {PU} PU\nE $\in$ [10,600]\n$\eta$ $\in$ [1.5, 3]", xy = (0.4,0.4), xycoords='axes fraction')
    #hep.cms.text('Simulation')
